In [114]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pickle

In [2]:
import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback
import glob
import os
from pprint import pprint
import simplejson as json
import requests
import time
from IPython.display import display
import psycopg2 as ps
import pandas as pd
import numpy as np


In [3]:
URI="dublinbike.cckaxoirozab.eu-west-1.rds.amazonaws.com"
PORT=3306
DB="dublinbike"
USER="comp30830"
PASSWORD="dublinbike"

In [4]:
engine=create_engine("mysql+mysqldb://{}:{}@{}:{}/{}".format(USER,PASSWORD,URI,PORT,DB),echo=True)

In [5]:
data = pd.read_sql('select * from station', engine)
print(data)

2023-04-04 10:06:34,800 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-04-04 10:06:34,802 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-04 10:06:34,818 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-04-04 10:06:34,819 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-04 10:06:34,827 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-04-04 10:06:34,828 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-04 10:06:34,851 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %s AND table_name = %s
2023-04-04 10:06:34,852 INFO sqlalchemy.engine.Engine [generated in 0.00113s] ('dublinbike', 'select * from station')
2023-04-04 10:06:34,869 INFO sqlalchemy.engine.Engine select * from station
2023-04-04 10:06:34,870 INFO sqlalchemy.engine.Engine [raw sql] ()
                   address  banking  bonus contract_name  \
0            Clarendon Row        0      0        dublin   
1       Blessington Street        0      0 

In [6]:
df = pd.DataFrame(data)

In [7]:
df.to_csv('station.csv')

In [8]:
df=pd.read_csv("station.csv")

In [9]:
df

,Unnamed: 0,address,banking,bonus,contract_name,name,number,position_lat,position_lng,status
0,0,Clarendon Row,0,0,dublin,CLARENDON ROW,1,53.340927,-6.262501,OPEN
1,1,Blessington Street,0,0,dublin,BLESSINGTON STREET,2,53.356769,-6.268140,OPEN
2,2,Bolton Street,0,0,dublin,BOLTON STREET,3,53.351182,-6.269859,OPEN
3,3,Greek Street,0,0,dublin,GREEK STREET,4,53.346874,-6.272976,OPEN
4,4,Charlemont Street,0,0,dublin,CHARLEMONT PLACE,5,53.330662,-6.260177,OPEN
...,...,...,...,...,...,...,...,...,...,...
109,109,Merrion Square South,0,0,dublin,MERRION SQUARE SOUTH,113,53.338614,-6.248606,OPEN
110,110,Wilton Terrace (Park),0,0,dublin,WILTON TERRACE (PARK),114,53.333653,-6.248345,OPEN
111,111,Killarney Street,0,0,dublin,KILLARNEY STREET,115,53.354845,-6.247579,OPEN
112,112,Broadstone,0,0,dublin,BROADSTONE,116,53.354700,-6.272314,OPEN


In [10]:
availability = pd.read_sql('select * from availability', engine)
print(availability)

2023-04-04 10:06:34,951 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %s AND table_name = %s
2023-04-04 10:06:34,951 INFO sqlalchemy.engine.Engine [cached since 0.1008s ago] ('dublinbike', 'select * from availability')
2023-04-04 10:06:35,316 INFO sqlalchemy.engine.Engine select * from availability
2023-04-04 10:06:35,319 INFO sqlalchemy.engine.Engine [raw sql] ()
        number  available_bikes  available_bikes_stands  \
0            1               17                      14   
1            1                0                      31   
2            1                0                      31   
3            1                0                      31   
4            1                0                      31   
...        ...              ...                     ...   
309847     117               39                       1   
309848     117               39                       1   
309849     117               39                   

In [11]:
df= pd.DataFrame(availability)

In [12]:
df.to_csv('availability.csv')

In [13]:
df

,number,available_bikes,available_bikes_stands,last_update
0,1,17,14,2023
1,1,0,31,2023-03-23 01:13:12.639790
2,1,0,31,2023-03-23 01:18:13.417911
3,1,0,31,2023-03-23 01:26:07.467583
4,1,0,31,2023-03-23 01:31:08.366535
...,...,...,...,...
309847,117,39,1,2023-04-04 08:44:11.253792
309848,117,39,1,2023-04-04 08:49:12.381701
309849,117,39,1,2023-04-04 08:54:13.516158
309850,117,39,1,2023-04-04 08:59:14.637067


In [14]:
weather = pd.read_sql('select * from weather', engine)
print(weather)

2023-04-04 10:06:39,453 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %s AND table_name = %s
2023-04-04 10:06:39,453 INFO sqlalchemy.engine.Engine [cached since 4.603s ago] ('dublinbike', 'select * from weather')
2023-04-04 10:06:39,507 INFO sqlalchemy.engine.Engine select * from weather
2023-04-04 10:06:39,508 INFO sqlalchemy.engine.Engine [raw sql] ()
    weather    temp  feel_temp  min_temp  max_temp  pressure  humidity  \
0      Rain  281.19     277.21    280.51    281.69       987        76   
1      Rain  281.19     277.21    280.51    281.69       987        76   
2    Clouds  280.31     275.78    279.68    280.58       987        83   
3      Rain  280.77     277.13    279.51    281.48       986        86   
4    Clouds  280.56     276.86    280.00    281.47       986        84   
..      ...     ...        ...       ...       ...       ...       ...   
412    Rain  280.85     277.80    280.36    281.68      1024        81   


In [15]:
df = pd.DataFrame(weather)

In [16]:
df.to_csv('weather.csv')

In [17]:
df

,weather,temp,feel_temp,min_temp,max_temp,pressure,humidity,visibility,wind_speed,wind_deg,time
0,Rain,281.19,277.21,280.51,281.69,987,76,10000,8.23,200,2023-03-23 01:22:41.387915
1,Rain,281.19,277.21,280.51,281.69,987,76,10000,8.23,200,2023-03-23 01:26:44.796587
2,Clouds,280.31,275.78,279.68,280.58,987,83,10000,9.26,220,2023-03-23 02:26:44.940925
3,Rain,280.77,277.13,279.51,281.48,986,86,10000,6.69,210,2023-03-23 03:26:45.041766
4,Clouds,280.56,276.86,280.00,281.47,986,84,10000,6.69,200,2023-03-23 04:26:45.163174
...,...,...,...,...,...,...,...,...,...,...,...
412,Rain,280.85,277.80,280.36,281.68,1024,81,10000,5.14,200,2023-04-04 06:27:16.238759
413,Rain,281.02,279.40,280.51,281.48,1024,81,10000,2.57,140,2023-04-04 07:08:07.820620
414,Rain,281.36,279.46,280.55,282.04,1024,80,10000,3.09,150,2023-04-04 07:27:16.338364
415,Rain,281.51,279.98,281.11,281.80,1024,80,10000,2.57,140,2023-04-04 08:08:07.908650


## clean data

### stations

In [18]:
df=pd.read_csv("station.csv")

In [19]:
df.dtypes

Unnamed: 0         int64
address           object
banking            int64
bonus              int64
contract_name     object
name              object
number             int64
position_lat     float64
position_lng     float64
status            object
dtype: object

In [20]:
df.isnull().sum()

Unnamed: 0       0
address          0
banking          0
bonus            0
contract_name    0
name             0
number           0
position_lat     0
position_lng     0
status           0
dtype: int64

In [21]:
# test_1=df[(df['last_update']=='2023')]
# print("Number of rows failing the test: ", test_1.shape[0])

In [22]:
# df=df.drop(test_1)

In [23]:
df_availability =pd.read_csv("availability.csv",index_col=0)

In [24]:
df_availability.isnull().sum()

number                    0
available_bikes           0
available_bikes_stands    0
last_update               0
dtype: int64

In [25]:
df_availability.tail(10)

,number,available_bikes,available_bikes_stands,last_update
309842,117,39,0,2023-04-04 08:19:06.863749
309843,117,40,0,2023-04-04 08:24:07.725445
309844,117,40,0,2023-04-04 08:29:08.477952
309845,117,40,0,2023-04-04 08:34:09.478991
309846,117,40,0,2023-04-04 08:39:10.277337
309847,117,39,1,2023-04-04 08:44:11.253792
309848,117,39,1,2023-04-04 08:49:12.381701
309849,117,39,1,2023-04-04 08:54:13.516158
309850,117,39,1,2023-04-04 08:59:14.637067
309851,117,39,1,2023-04-04 09:04:15.414279


In [26]:
df_availability = df_availability[df_availability['last_update']!='2023'].reset_index(drop=True)

In [27]:
# df_availability.last_update[1][0:13]+':00:00'
result=[]
for i in range(len(df_availability)):
    result.append(df_availability.last_update[i][0:13]+':00:00')

In [28]:
df_availability["hour"]=result

In [29]:
df_availability.head(10)

,number,available_bikes,available_bikes_stands,last_update,hour
0,1,0,31,2023-03-23 01:13:12.639790,2023-03-23 01:00:00
1,1,0,31,2023-03-23 01:18:13.417911,2023-03-23 01:00:00
2,1,0,31,2023-03-23 01:26:07.467583,2023-03-23 01:00:00
3,1,0,31,2023-03-23 01:31:08.366535,2023-03-23 01:00:00
4,1,0,31,2023-03-23 01:36:09.196093,2023-03-23 01:00:00
5,1,0,31,2023-03-23 01:41:09.973804,2023-03-23 01:00:00
6,1,0,31,2023-03-23 01:46:10.703312,2023-03-23 01:00:00
7,1,0,31,2023-03-23 01:51:11.487183,2023-03-23 01:00:00
8,1,0,31,2023-03-23 01:56:12.310439,2023-03-23 01:00:00
9,1,0,31,2023-03-23 02:01:13.054742,2023-03-23 02:00:00


In [30]:
df_weather=pd.read_csv("weather.csv",index_col=0)

In [31]:
result=[]
for i in range(len(df_weather)):
    result.append(df_weather.time[i][0:13]+':00:00')

In [32]:
df_weather["hour"]=result

In [33]:
df_weather.head(5)

,weather,temp,feel_temp,min_temp,max_temp,pressure,humidity,visibility,wind_speed,wind_deg,time,hour
0,Rain,281.19,277.21,280.51,281.69,987,76,10000,8.23,200,2023-03-23 01:22:41.387915,2023-03-23 01:00:00
1,Rain,281.19,277.21,280.51,281.69,987,76,10000,8.23,200,2023-03-23 01:26:44.796587,2023-03-23 01:00:00
2,Clouds,280.31,275.78,279.68,280.58,987,83,10000,9.26,220,2023-03-23 02:26:44.940925,2023-03-23 02:00:00
3,Rain,280.77,277.13,279.51,281.48,986,86,10000,6.69,210,2023-03-23 03:26:45.041766,2023-03-23 03:00:00
4,Clouds,280.56,276.86,280.00,281.47,986,84,10000,6.69,200,2023-03-23 04:26:45.163174,2023-03-23 04:00:00


In [34]:
df_availability_new=df_availability.groupby(["number","hour"])["available_bikes"].mean()

In [35]:
df_availability_new=pd.DataFrame(df_availability_new)

In [36]:
df_availability_new = df_availability_new.reset_index()

In [37]:
df_availability_new.head(10)

,number,hour,available_bikes
0,1,2023-03-23 01:00:00,0.000000
1,1,2023-03-23 02:00:00,0.000000
2,1,2023-03-23 03:00:00,0.000000
3,1,2023-03-23 04:00:00,0.000000
4,1,2023-03-23 05:00:00,0.000000
5,1,2023-03-23 06:00:00,1.000000
6,1,2023-03-23 07:00:00,7.166667
7,1,2023-03-23 08:00:00,13.416667
8,1,2023-03-23 09:00:00,21.416667
9,1,2023-03-23 10:00:00,21.500000


In [38]:
df_availability_new=df_availability_new.merge(df_weather,on='hour')

In [39]:
pd.set_option('display.max_rows', None)
df_availability[['available_bikes','last_update']].set_index(pd.DatetimeIndex(df_availability.last_update)).resample(rule='1H').mean()



/var/folders/wr/7y3zysbj7v98hljgp945drfc0000gn/T/ipykernel_88680/564370761.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_availability[['available_bikes','last_update']].set_index(pd.DatetimeIndex(df_availability.last_update)).resample(rule='1H').mean()


,available_bikes
last_update,
2023-03-23 01:00:00,12.052632
2023-03-23 02:00:00,12.059942
2023-03-23 03:00:00,12.061404
2023-03-23 04:00:00,12.061404
2023-03-23 05:00:00,11.935673
2023-03-23 06:00:00,11.548977
2023-03-23 07:00:00,11.152778
2023-03-23 08:00:00,10.064327
2023-03-23 09:00:00,10.671053


In [40]:
# test_1=df[(df['last_update']=='2023')]
# print("Number of rows failing the test: ", test_1.shape[0])

In [41]:
# df=df.drop(test_1)

In [42]:
df_weather.isnull().sum()

weather       0
temp          0
feel_temp     0
min_temp      0
max_temp      0
pressure      0
humidity      0
visibility    0
wind_speed    0
wind_deg      0
time          0
hour          0
dtype: int64

In [43]:
pd.set_option('display.max_rows', None)
df_weather.set_index(pd.DatetimeIndex(df_weather.time)).resample(rule='1H').mean()

/var/folders/wr/7y3zysbj7v98hljgp945drfc0000gn/T/ipykernel_88680/384353120.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_weather.set_index(pd.DatetimeIndex(df_weather.time)).resample(rule='1H').mean()


,temp,feel_temp,min_temp,max_temp,pressure,humidity,visibility,wind_speed,wind_deg
time,,,,,,,,,
2023-03-23 01:00:00,281.190,277.210,280.510,281.690,987.0,76.0,10000.0,8.230,200.0
2023-03-23 02:00:00,280.310,275.780,279.680,280.580,987.0,83.0,10000.0,9.260,220.0
2023-03-23 03:00:00,280.770,277.130,279.510,281.480,986.0,86.0,10000.0,6.690,210.0
2023-03-23 04:00:00,280.560,276.860,280.000,281.470,986.0,84.0,10000.0,6.690,200.0
2023-03-23 05:00:00,281.090,277.220,280.510,281.690,986.0,83.0,10000.0,7.720,210.0
2023-03-23 06:00:00,281.090,277.080,280.510,281.690,986.0,81.0,10000.0,8.230,210.0
2023-03-23 07:00:00,281.510,277.360,280.510,282.250,986.0,81.0,10000.0,9.260,210.0
2023-03-23 08:00:00,282.730,279.970,282.020,283.150,986.0,78.0,10000.0,5.660,190.0
2023-03-23 09:00:00,284.010,283.080,282.680,284.580,986.0,74.0,10000.0,10.290,210.0


In [44]:
# df_availability['last_update'] = pd.to_datetime(df_availability['last_update'])
# df_weather['time'] = pd.to_datetime(df_weather['time'])

In [45]:
df_availability.dtypes

number                     int64
available_bikes            int64
available_bikes_stands     int64
last_update               object
hour                      object
dtype: object

In [46]:
# df_availability['last_update'] = pd.to_datetime(df_availability['last_update'], format='%Y-%m-%d %H:%M:%S')
# df_weather['time'] = pd.to_datetime(df_weather['time'], format='%Y-%m-%d %H:%M:%S')

In [47]:
# df_availability.set_index('last_update', inplace=True)
# df_weather.set_index('time', inplace=True)

In [48]:
df_weather.dtypes

weather        object
temp          float64
feel_temp     float64
min_temp      float64
max_temp      float64
pressure        int64
humidity        int64
visibility      int64
wind_speed    float64
wind_deg        int64
time           object
hour           object
dtype: object

In [49]:
# print(df_availability.index)
# print(df_weather.index)

In [50]:
# df_weather_resampled = weather.sort_values(by='time')
# df_weather_resampled = df_weather_resampled.set_index('time')
# df_weather_resampled = df_weather_resampled.resample('5T').ffill()

In [51]:
df_availability_new["hour"] = df_availability_new["hour"].astype("datetime64[ns]")

In [52]:
df_availability_new['year']=pd.DatetimeIndex(df_availability_new['hour']).year
df_availability_new['month']=pd.DatetimeIndex(df_availability_new['hour']).month
df_availability_new['day']=pd.DatetimeIndex(df_availability_new['hour']).day
df_availability_new['hours']=pd.DatetimeIndex(df_availability_new['hour']).hour

In [53]:
# df_availability_new["hour"]=pd.to_datetime(df_availability_new['hour'])


In [54]:
# df_availability_new['hour_float'] = df_availability_new['hour'].apply(lambda x: x.timestamp())


In [55]:
# df_availability_new["temp"]=df_availability_new["temp"].astype("object")
# df_availability_new["humidity"]=df_availability_new["humidity"].astype("object")
# df_availability_new["wind_speed"]=df_availability_new["wind_speed"].astype("object")
# df_availability_new["weather"]=df_availability_new["weather"].astype("object")


In [56]:
# df_availability_new["available_bikes"].astype("object")


In [57]:
df_availability_new.dtypes

number                      int64
hour               datetime64[ns]
available_bikes           float64
weather                    object
temp                      float64
feel_temp                 float64
min_temp                  float64
max_temp                  float64
pressure                    int64
humidity                    int64
visibility                  int64
wind_speed                float64
wind_deg                    int64
time                       object
year                        int64
month                       int64
day                         int64
hours                       int64
dtype: object

In [63]:
cols=["number","month","day","hours","temp","humidity","wind_speed","weather"]

In [75]:
df_availability_new["weather"].unique()

array(['Rain', 'Clouds', 'Clear', 'Mist', 'Fog', 'Drizzle'], dtype=object)

In [106]:
enc = OneHotEncoder(handle_unknown='ignore')
X = [['Rain', 1], ['Clouds', 2], ['Clear', 3],['Mist', 4],['Fog', 5],['Drizzle', 6]]
enc.fit(X)
OneHotEncoder(handle_unknown='ignore')
# enc.categories_
# enc.categories_[np.array(['Rain', 'Clouds', 'Clear', 'Mist', 'Fog', 'Drizzle'], dtype=object), np.array([1, 2, 3,4,5,6], dtype=object)]
enc.transform([['Rain', 1], ['Clouds', 2], ['Clear', 3],['Mist', 4],['Fog', 5],['Drizzle', 6]]).toarray()



array([[0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [98]:
enc = OneHotEncoder(handle_unknown='ignore')
weather_array = df_availability_new['weather']

In [102]:
test_array = ['Rain','Clouds','Clear','Mist', 'Fog','Drizzle']
from sklearn.preprocessing import LabelBinarizer
y = LabelBinarizer().fit_transform(test_array)
print(y)

[[0 0 0 0 0 1]
 [0 1 0 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 1 0]
 [0 0 0 1 0 0]
 [0 0 1 0 0 0]]


In [108]:
X = df_availability_new[cols]
y = df_availability_new['available_bikes']


In [109]:
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [110]:
a= LinearRegression() # choose your own model here...
a.fit(X_train, y_train)


LinearRegression()

In [115]:
with open('model.pkl', 'wb') as handle:
    pickle.dump(a, handle, pickle.HIGHEST_PROTOCOL)

In [116]:
result=a.predict(X_test)

In [117]:
print(result)

[11.45088053 11.3773269  12.00341101 ... 11.78583857 11.58522115
 11.43274175]
